# Flow Predictor with MLflow Integration

This notebook implements MLflow tracking for the flow predictor model.

In [ ]:
# Import section - add MLflow imports
import keras
from keras.models import Model
from keras import backend as K
from keras.layers import (
    Input, 
    concatenate, 
    Conv2D,
    MaxPooling2D,
    Conv2DTranspose,
    ZeroPadding2D
)
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, Flatten

import vtk
from vtm_data import VTK_data
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import mlflow
import mlflow.keras

# Create experiment
mlflow.set_experiment("Flow_Predictor_Training")

# Enable MLflow autologging
mlflow.keras.autolog(
    log_models=True,
    log_model_signatures=True,
    log_input_examples=True,
    registered_model_name="flow_predictor"
)

In [ ]:
# training params
batch_size = 32
epochs = 50 # number of times through training set

In [ ]:
# load dataset
class VTK_data:
    def __init__(self, base_path, split_ratio=0.8):
        self.base_path = Path(base_path)
        self.data = []
        self.geometries = []
        self.steady_flows = []
        self.split_ratio = split_ratio
        self.split_line = 0
        
    def load_data(self):
        for dirpath, dirnames, filenames in os.walk(self.base_path):
            for filename in filenames:
                if filename.endswith('.vtm'):
                    full_path = Path(dirpath) / filename
                    try:
                        data = self._load_single_file(full_path)
                        if data is not None:
                            print(f"\nProcessing file: {filename}")
                            print(f"Full path: {full_path}")
                            
                            if 'geometry' in filename:
                                print(f"Classified as geometry file")
                                self.geometries.append(data)
                            elif 'cylinder2d_iT' in filename:
                                print(f"Classified as flow file")
                                self.steady_flows.append(data)
                            else:
                                print(f"Skipping file: {filename}")
                    except Exception as e:
                        print(f"Error loading {full_path}: {str(e)}")
        
        print("\nLoaded data summary:")
        print(f"Number of geometry files: {len(self.geometries)}")
        if self.geometries:
            print(f"Shape of first geometry: {np.array(self.geometries[0]).shape}")
        
        print(f"Number of flow files: {len(self.steady_flows)}")
        if self.steady_flows:
            print(f"Shape of first flow: {np.array(self.steady_flows[0]).shape}")
        
        total_samples = len(self.geometries)
        self.split_line = int(total_samples * self.split_ratio)
        
        return self.geometries, self.steady_flows
    
    def _load_single_file(self, file_path):
        reader = vtk.vtkXMLMultiBlockDataReader()
        reader.SetFileName(str(file_path))
        reader.Update()
        
        if reader.GetErrorCode() != 0:
            raise RuntimeError(f"Error reading file")
        
        data = reader.GetOutput()
        if data is None:
            raise RuntimeError("No data read from file")
            
        data_iterator = data.NewIterator()
        img_data = data_iterator.GetCurrentDataObject()
        
        if img_data is None:
            raise RuntimeError("No image data found in file")
        
        if hasattr(img_data, 'GetProducerPort'):
            producer = img_data.GetProducerPort()
            if producer:
                producer.Update()
        elif hasattr(img_data, 'GetSource'):
            source = img_data.GetSource()
            if source:
                source.Update()
                
        point_data = img_data.GetPointData()
        array_data = point_data.GetArray(0)
        array_data = vtk.util.numpy_support.vtk_to_numpy(array_data)
        
        return array_data

# Create instance and load data
base_directory = "/mnt/data/cfd-ml-examples/sumulation"
dataset = VTK_data(base_directory)

# Load the data
geometries, steady_flows = dataset.load_data()

In [ ]:
# get train and test split
train_geometries = dataset.geometries[0:dataset.split_line]
train_steady_flows = dataset.steady_flows[0:dataset.split_line]
test_geometries = dataset.geometries[dataset.split_line:-1]
test_steady_flows = dataset.steady_flows[dataset.split_line:-1]

print(f"Training set size: {len(train_geometries)} samples")
print(f"Test set size: {len(test_geometries)} samples")